# GOOGLE MAP API

# IMPORT LIBS

In [ ]:
import pandas as pd
import json
import requests
import time

# OPEN DATASET

In [ ]:
df = pd.read_csv('../../DATA/PREPROCESSING/prepro_01_geo.csv')
df

# GET MY API KEY

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
API_key = get_keys("../../ASSETS/GMAP/api_key.json")
google_key = list(API_key.values())[0]

# GET THE DRIVING AN DURATION DISTANCE AND INSERT INTO LIST

In [ ]:
lat_origin = df['pickup_latitude'].tolist()
long_origin = df['pickup_longitude'].tolist()
lat_destination = df['dropoff_latitude'].tolist()
long_destination = df['dropoff_longitude'].tolist()

distance = []
duration = []
drop = []
for i in range(len(long_destination)):
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins={lat_origin[i]},{long_origin[i]}&destinations={lat_destination[i]}%2C{long_destination[i]}&key={google_key}"
    print(url)
    r=requests.get(url)
    print(r)
    data = r.json()
    print(data)
    try:
        print(data['rows'][0]['elements'][0]['distance']['text'])
        distance.append(data['rows'][0]['elements'][0]['distance']['text'])
        print(data['rows'][0]['elements'][0]['duration']['text'])
        duration.append(data['rows'][0]['elements'][0]['duration']['text'])
    except:
        drop.append([lat_origin[i], long_origin[i], lat_destination[i], long_destination[i]])
        pass
    time.sleep(0.5)

# CLEAN THE RESULT AND INSERT INTO DF

In [ ]:
distance_cleaned = []
duration_cleaned = []
for i in range(len(distance)):
    distance_clean = distance[i].replace(' mi', '').replace(' ft', '')
    float_distance_clean = float(distance_clean)
    distance_cleaned.append(float_distance_clean)
    duration_clean = duration[i].replace(' mins', '').replace(' min', '')
    float_duration_clean = float(duration_clean)
    duration_cleaned.append(float_duration_clean)


df['Driving distance (miles)'] = distance_cleaned
df['Driving duration'] = duration_cleaned
df.head()

# EXPORT DF

In [ ]:
df.to_csv('../../DATA/PREPROCESSING/prepro_01a_geo_gmap.csv', index=False)